In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10620221413657099301, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5386302988389560644
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [11]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=DenseNet201(weights="imagenet", include_top=False, pooling = "avg")

In [12]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "DenseNet201_descriptors"

In [13]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [14]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [15]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [16]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [17]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [18]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [19]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [20]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [21]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_2': 0.7, 'beta_1': 0.9, 'lr': 0.001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.6226 - acc: 0.7784 - val_loss: 0.8815 - val_acc: 0.6391
Epoch 2/15
 - 4s - loss: 0.5613 - acc: 0.8068 - val_loss: 0.7934 - val_acc: 0.6794
Epoch 3/15
 - 4s - loss: 0.5771 - acc: 0.7964 - val_loss: 0.8180 - val_acc: 0.6945
Epoch 4/15
 - 4s - loss: 0.6130 - acc: 0.7822 - val_loss: 0.9697 - val_acc: 0.6695
Epoch 5/15
 - 4s - loss: 0.6246 - acc: 0.7779 - val_loss: 0.9125 - val_acc: 0.6864
Epoch 6/15
 - 4s - loss: 0.6473 - acc: 0.7693 - val_loss: 0.9171 - val_acc: 0.6983
Epoch 7/15
 - 4s - loss: 0.6462 - acc: 0.7693 - val_loss: 1.0405 - val_acc: 0.6816
Epoch 8/15
 - 4s - loss: 0.6803 - acc: 0.7579 - val_loss: 0.9250 - val_acc: 0.6693
Epoch 9/15
 - 4s - loss: 0.7260 - acc: 0.7418 - val_loss: 1.0293 - val_acc: 0.6617
Epoch 00009: early stopping
14958/14958 [==============================] - 1s 35us/step
Validation Accuracy: 66.1719%
Validation Loss: 1.0

Epoch 5/15
 - 4s - loss: 0.7088 - acc: 0.7461 - val_loss: 0.9811 - val_acc: 0.5935
Epoch 6/15
 - 4s - loss: 0.6756 - acc: 0.7584 - val_loss: 0.9636 - val_acc: 0.6017
Epoch 7/15
 - 4s - loss: 0.6479 - acc: 0.7712 - val_loss: 0.9482 - val_acc: 0.6112
Epoch 8/15
 - 4s - loss: 0.6327 - acc: 0.7775 - val_loss: 0.9350 - val_acc: 0.6174
Epoch 9/15
 - 4s - loss: 0.6124 - acc: 0.7834 - val_loss: 0.9204 - val_acc: 0.6231
Epoch 10/15
 - 4s - loss: 0.5964 - acc: 0.7884 - val_loss: 0.9128 - val_acc: 0.6282
Epoch 11/15
 - 4s - loss: 0.5853 - acc: 0.7943 - val_loss: 0.9053 - val_acc: 0.6330
Epoch 12/15
 - 4s - loss: 0.5804 - acc: 0.7995 - val_loss: 0.9031 - val_acc: 0.6372
Epoch 13/15
 - 4s - loss: 0.5678 - acc: 0.8038 - val_loss: 0.9054 - val_acc: 0.6369
Epoch 14/15
 - 4s - loss: 0.5580 - acc: 0.8077 - val_loss: 0.8988 - val_acc: 0.6407
Epoch 15/15
 - 4s - loss: 0.5511 - acc: 0.8094 - val_loss: 0.8932 - val_acc: 0.6452
14958/14958 [==============================] - 1s 35us/step
Validation Accuracy: 

Parameters testing:  {'beta_2': 0.4, 'beta_1': 0.5, 'lr': 0.01}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.3009 - acc: 0.6358 - val_loss: 1.8128 - val_acc: 0.4469
Epoch 2/15
 - 4s - loss: 1.5272 - acc: 0.6522 - val_loss: 2.0863 - val_acc: 0.4775
Epoch 3/15
 - 4s - loss: 1.5897 - acc: 0.6564 - val_loss: 2.4640 - val_acc: 0.5072
Epoch 4/15
 - 4s - loss: 1.6497 - acc: 0.6714 - val_loss: 2.8841 - val_acc: 0.5101
Epoch 5/15
 - 4s - loss: 1.7177 - acc: 0.6811 - val_loss: 2.2689 - val_acc: 0.5401
Epoch 6/15
 - 4s - loss: 1.7253 - acc: 0.6838 - val_loss: 2.9138 - val_acc: 0.5332
Epoch 7/15
 - 4s - loss: 1.7663 - acc: 0.6921 - val_loss: 2.4432 - val_acc: 0.5695
Epoch 8/15
 - 4s - loss: 1.7023 - acc: 0.6888 - val_loss: 2.9872 - val_acc: 0.5097
Epoch 9/15
 - 4s - loss: 1.8088 - acc: 0.6974 - val_loss: 2.5167 - val_acc: 0.5584
Epoch 10/15
 - 4s - loss: 1.8637 - acc: 0.6962 - val_loss: 2.7405 - val_acc: 0.5965
Epoch 11/15
 - 4s - loss: 1.8890 - acc: 0.7045 - val_lo

Parameters testing:  {'beta_2': 0.99, 'beta_1': 0.9, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.7771 - acc: 0.7213 - val_loss: 0.8753 - val_acc: 0.6399
Epoch 2/15
 - 4s - loss: 0.5217 - acc: 0.8154 - val_loss: 0.8348 - val_acc: 0.6556
Epoch 3/15
 - 4s - loss: 0.4586 - acc: 0.8390 - val_loss: 0.8013 - val_acc: 0.6759
Epoch 4/15
 - 4s - loss: 0.4256 - acc: 0.8518 - val_loss: 0.8151 - val_acc: 0.6697
Epoch 5/15
 - 4s - loss: 0.3981 - acc: 0.8593 - val_loss: 0.7867 - val_acc: 0.6897
Epoch 6/15
 - 4s - loss: 0.3798 - acc: 0.8680 - val_loss: 0.7955 - val_acc: 0.6881
Epoch 7/15
 - 4s - loss: 0.3621 - acc: 0.8750 - val_loss: 0.7770 - val_acc: 0.7014
Epoch 8/15
 - 4s - loss: 0.3506 - acc: 0.8787 - val_loss: 0.7902 - val_acc: 0.7020
Epoch 9/15
 - 4s - loss: 0.3367 - acc: 0.8842 - val_loss: 0.7939 - val_acc: 0.6991
Epoch 10/15
 - 4s - loss: 0.3283 - acc: 0.8865 - val_loss: 0.8194 - val_acc: 0.6929
Epoch 11/15
 - 4s - loss: 0.3189 - acc: 0.8896 - val

Test Accuracy: 0.8350272499631757
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.5, 'beta_1': 0.2, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.8114 - acc: 0.7081 - val_loss: 0.9273 - val_acc: 0.6252
Epoch 2/15
 - 4s - loss: 0.5793 - acc: 0.7996 - val_loss: 0.8987 - val_acc: 0.6467
Epoch 3/15
 - 4s - loss: 0.5291 - acc: 0.8219 - val_loss: 0.8963 - val_acc: 0.6604
Epoch 4/15
 - 4s - loss: 0.5067 - acc: 0.8336 - val_loss: 0.8842 - val_acc: 0.6746
Epoch 5/15
 - 4s - loss: 0.4959 - acc: 0.8423 - val_loss: 0.8876 - val_acc: 0.6887
Epoch 6/15
 - 4s - loss: 0.4840 - acc: 0.8483 - val_loss: 0.8972 - val_acc: 0.6859
Epoch 7/15
 - 4s - loss: 0.4822 - acc: 0.8539 - val_loss: 0.9471 - val_acc: 0.6920
Epoch 8/15
 - 4s - loss: 0.4814 - acc: 0.8565 - val_loss: 0.9670 - val_acc: 0.6843
Epoch 9/15
 - 4s - loss: 0.4842 - a

...

Epoch 00007: early stopping
14958/14958 [==============================] - 2s 134us/step
Validation Accuracy: 71.9882%
Validation Loss: 0.8530260795061132
Test Accuracy: 0.8482839888054206
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.99, 'beta_1': 0.7, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 0.8011 - acc: 0.7154 - val_loss: 0.8847 - val_acc: 0.6318
Epoch 2/15
 - 10s - loss: 0.5303 - acc: 0.8126 - val_loss: 0.8747 - val_acc: 0.6388
Epoch 3/15
 - 10s - loss: 0.4632 - acc: 0.8373 - val_loss: 0.7958 - val_acc: 0.6761
Epoch 4/15
 - 10s - loss: 0.4267 - acc: 0.8510 - val_loss: 0.8332 - val_acc: 0.6723
Epoch 5/15
 - 9s - loss: 0.4019 - acc: 0.8588 - val_loss: 0.8376 - val_acc: 0.6739
Epoch 6/15
 - 9s - loss: 0.3800 - acc: 0.8684 - val_loss: 0.8257 - val_acc: 0.6810
Epoch 7/15
 - 9s - loss: 0.3656 - acc: 0.

Parameters testing:  {'beta_2': 0.995, 'beta_1': 0.4, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 0.7860 - acc: 0.7214 - val_loss: 0.8961 - val_acc: 0.6329
Epoch 2/15
 - 10s - loss: 0.5248 - acc: 0.8164 - val_loss: 0.8114 - val_acc: 0.6714
Epoch 3/15
 - 10s - loss: 0.4615 - acc: 0.8372 - val_loss: 0.8166 - val_acc: 0.6684
Epoch 4/15
 - 10s - loss: 0.4220 - acc: 0.8516 - val_loss: 0.7813 - val_acc: 0.6877
Epoch 5/15
 - 10s - loss: 0.3976 - acc: 0.8624 - val_loss: 0.7741 - val_acc: 0.6919
Epoch 6/15
 - 10s - loss: 0.3740 - acc: 0.8706 - val_loss: 0.8167 - val_acc: 0.6818
Epoch 7/15
 - 9s - loss: 0.3593 - acc: 0.8750 - val_loss: 0.8284 - val_acc: 0.6816
Epoch 8/15
 - 10s - loss: 0.3462 - acc: 0.8798 - val_loss: 0.7672 - val_acc: 0.7115
Epoch 9/15
 - 9s - loss: 0.3374 - acc: 0.8818 - val_loss: 0.7699 - val_acc: 0.7032
Epoch 10/15
 - 9s - loss: 0.3263 - acc: 0.8862 - val_loss: 0.7978 - val_acc: 0.6988
Epoch 11/15
 - 10s - loss: 0.3199 - acc: 0.8

 - 10s - loss: 0.5475 - acc: 0.8244 - val_loss: 0.7958 - val_acc: 0.7061
Epoch 3/15
 - 10s - loss: 0.5677 - acc: 0.8205 - val_loss: 0.9244 - val_acc: 0.7093
Epoch 4/15
 - 10s - loss: 0.5982 - acc: 0.8172 - val_loss: 0.9085 - val_acc: 0.7143
Epoch 5/15
 - 10s - loss: 0.6277 - acc: 0.8139 - val_loss: 1.2069 - val_acc: 0.6774
Epoch 6/15
 - 10s - loss: 0.6554 - acc: 0.8101 - val_loss: 1.1346 - val_acc: 0.6845
Epoch 7/15
 - 10s - loss: 0.6915 - acc: 0.8031 - val_loss: 1.1709 - val_acc: 0.6754
Epoch 00007: early stopping
14958/14958 [==============================] - 2s 146us/step
Validation Accuracy: 67.5425%
Validation Loss: 1.1709079614874127
Test Accuracy: 0.8166151126822802
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.5, 'beta_1': 0.9, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 0.6849 - acc: 0.7560 - va

Epoch 11/15
 - 10s - loss: 0.5861 - acc: 0.7928 - val_loss: 0.9096 - val_acc: 0.6320
Epoch 12/15
 - 10s - loss: 0.5772 - acc: 0.7975 - val_loss: 0.8984 - val_acc: 0.6391
Epoch 13/15
 - 10s - loss: 0.5658 - acc: 0.8013 - val_loss: 0.8918 - val_acc: 0.6428
Epoch 14/15
 - 10s - loss: 0.5593 - acc: 0.8045 - val_loss: 0.8864 - val_acc: 0.6463
Epoch 15/15
 - 10s - loss: 0.5544 - acc: 0.8098 - val_loss: 0.8855 - val_acc: 0.6473
14958/14958 [==============================] - 3s 196us/step
Validation Accuracy: 64.7279%
Validation Loss: 0.8855474775950688
Test Accuracy: 0.7735307114449845
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.8, 'beta_1': 0.8, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 0.7742 - acc: 0.7223 - val_loss: 0.8884 - val_acc: 0.6369
Epoch 2/15
 - 10s - loss: 0.5309 - acc: 0.8115 - val_loss: 0.86

Epoch 4/15
 - 10s - loss: 2.0947 - acc: 0.2113 - val_loss: 1.7978 - val_acc: 0.1984
Epoch 00004: early stopping
14958/14958 [==============================] - 3s 176us/step
Validation Accuracy: 19.8422%
Validation Loss: 1.7978168434917585
Test Accuracy: 0.24841655619384298
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.995, 'beta_1': 0.1, 'lr': 0.001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 0.5811 - acc: 0.7931 - val_loss: 0.7686 - val_acc: 0.6931
Epoch 2/15
 - 10s - loss: 0.4400 - acc: 0.8457 - val_loss: 0.8178 - val_acc: 0.6876
Epoch 3/15
 - 10s - loss: 0.4011 - acc: 0.8552 - val_loss: 0.7989 - val_acc: 0.6954
Epoch 4/15
 - 10s - loss: 0.3833 - acc: 0.8659 - val_loss: 0.8567 - val_acc: 0.6875
Epoch 5/15
 - 10s - loss: 0.3643 - acc: 0.8716 - val_loss: 0.8297 - val_acc: 0.7207
Epoch 6/15
 - 10s - loss: 0.3523 - acc